In [160]:
##import libraries
import pulp 
import pandas as pd
import sys

##create data structures

cd = pd.read_excel('Transportation Optimization Problem.xlsx', "Cust_dmd_dictionary", index_col=[0]) #Customer demand file import
cdd = cd.to_dict(orient='index') #Customer demand dictionary
cdf = {k:cdd[k]['Demand'] for k in cdd} #customer demand flat

fc = pd.read_excel('Transportation Optimization Problem.xlsx', "Factory_cap_dictionary", index_col=[0]) #Factory cap file import
fcd = fc.to_dict(orient='index') #factory capacity dictionary
fcf = {k:fcd[k]['Capacity'] for k in fcd} #factory capacity flat

c = pd.read_excel('Transportation Optimization Problem.xlsx', "Cust_list")
cl = c.values.tolist()  #cust list
I = [item for sublist in cl for item in sublist] #cust list final/flat

p = pd.read_excel('Transportation Optimization Problem.xlsx', "Plant_list") 
pl = p.values.tolist() #plant list
J = [item for sublist in pl for item in sublist] #plant list final/flat

tc = pd.read_excel('Transportation Optimization Problem.xlsx', "Transp_cost", index_col=[0,1]) #transportation cost file import
tcd = tc.to_dict(orient='index') #transportation cost dictionary
tcf = {k:tcd[k]['Transportation_Cost'] for k in tcd} #transportation cost flat

pc = pd.read_excel('Transportation Optimization Problem.xlsx', "Plant_Fixed_Cost", index_col=[0]) #plant fixed cost file import
pcd = pc.to_dict(orient='index') #plant fixed cost dictionary
pcf = {k:pcd[k]['Cost'] for k in pcd} #plant fixed cost flat

##initialize decisision variables
x = pulp.LpVariable.dicts("Amt of goods", [(i,j) for i in I for j in J], lowBound= 0, cat='Continuous')

y = pulp.LpVariable.dicts("Plant Usage", pcf, lowBound= 0, upBound= 1, cat= 'Binary')


##declare objective function

objective = pulp.LpAffineExpression(e = [(x[i,j],tcf[i,j]) for i,j in x], name = 'TCF Objective function') + pulp.LpAffineExpression(e = [(y[l],pcf[l]) for l in y], name = 'PCF Objective function')

model = pulp.LpProblem(name = "Transportation cost minimization", 
                        sense = pulp.LpMinimize)
model += objective


##define constraints

## Constraint: sum of goods == customer demand
for i in I:
    tmpExpression = pulp.LpAffineExpression(e = [(x[i,j], 1) for j in J if (i,j) in x])
    tmpConstraint = pulp.LpConstraint(e = pulp.lpSum(tmpExpression),
        sense = pulp.LpConstraintEQ,                                
        rhs = cdf[i])
    model.addConstraint(tmpConstraint)
    
    
## Constraint: sum of goods <= factory capacity
for j in J:
    model += pulp.lpSum(x[(i,j)] for i in I) <= fcf[j] * y[j]       
    
    
## constraint for factory usage
for i in I:
    for j in J:
        model += x[(i,j)] <= cdf[i] * y[j]


    solver = model.solve()
    
    
    ##print all combination of decision variables
print('Objective function value =', pulp.value(model.objective))
print("All variables:")
for v in model.variables():
    print(v.name, "=", v.varValue)

Objective function value = 5610.0
All variables:
Amt_of_goods_(1,_1) = 0.0
Amt_of_goods_(1,_2) = 80.0
Amt_of_goods_(1,_3) = 0.0
Amt_of_goods_(2,_1) = 0.0
Amt_of_goods_(2,_2) = 270.0
Amt_of_goods_(2,_3) = 0.0
Amt_of_goods_(3,_1) = 0.0
Amt_of_goods_(3,_2) = 150.0
Amt_of_goods_(3,_3) = 100.0
Amt_of_goods_(4,_1) = 0.0
Amt_of_goods_(4,_2) = 0.0
Amt_of_goods_(4,_3) = 160.0
Amt_of_goods_(5,_1) = 0.0
Amt_of_goods_(5,_2) = 0.0
Amt_of_goods_(5,_3) = 180.0
Plant_Usage_1 = 0.0
Plant_Usage_2 = 1.0
Plant_Usage_3 = 1.0
